# Consignas

1)	Desarrollar al menos dos modelos de clasificación (decision tree, logistic regression) que tengan como target feature categoria_socio.
La target feature podrá asumir dos valores: 
●	Standard.
●	 Silver + Gold.
Aplicar las buenas prácticas en la construcción de modelos: dividir datos en conjunto de entrenamiento y validación, cross-validation, grid-search.
Obtener feature importance.

El objetivo es entender las diferencias entre las distintas categorías de afiliados. Cuáles son las variables más relevantes para diferenciarlos?
Las conclusiones obtenidas a partir de este modelo resultarán de gran importancia para el desarrollo de un modelo de recomendación.



In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling
import seaborn as sns
import ptitprince as pt
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [3]:
df_procesado=pd.read_csv('../Data/df_procesado.csv')
df_procesado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 31 columns):
 #   Column                                                                             Non-Null Count  Dtype  
---  ------                                                                             --------------  -----  
 0   id_consumo_encoded                                                                 2500 non-null   int64  
 1   id_socio_encoded                                                                   2500 non-null   int64  
 2   id_prestador_encoded                                                               2500 non-null   int64  
 3   calificacion_experiencia_encoded                                                   2500 non-null   float64
 4   id_zona_encoded                                                                    2500 non-null   int64  
 5   edad_encoded                                                                       2500 non-null   float

In [7]:
df_procesado.drop(['id_consumo_encoded','id_prestador_encoded','id_socio_encoded','id_zona_encoded'], axis = 1,inplace = True)
df_procesado['categoria_socio_unif']=df_procesado.categoria_socio_gold + dF_procesado.categoria_socio_silver + df_procesado.categoria_socio_standard

In [ ]:
model=LogisticRegression(random_state = 0, class_weigth = 'balanced')

parameters= {'penalty':['l1','l2'],
             'C':[0.0001,0.001,0.01,0.1,1,10,100],
             'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

cv=GridSearchCV(model , cv = 5 , njobs = -1, )
